In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import scipy.sparse
from sklearn import preprocessing
import xgboost as xgb
import pickle
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline

Read in just the testing and training data.

In [2]:
error = pd.read_pickle('scripts/cv_results/actions_e20/validate_err.p')
#error['mean'] = error.apply(np.mean, axis = 1)
error = error.astype(np.float64)
error['mean'] = error.mean(1)
error['std'] = error.std(1)

In [29]:
error

,0,0,0,0,0,mean,std
0,0.14032,0.14512,0.14570,0.14976,0.15203,0.146586,0.004047
1,0.13834,0.14374,0.14384,0.14755,0.15029,0.144752,0.004037
2,0.13719,0.14305,0.14227,0.14688,0.14859,0.143596,0.003970
3,0.13757,0.14336,0.14231,0.14590,0.14843,0.143514,0.003651
4,0.13755,0.14319,0.14179,0.14539,0.14798,0.143180,0.003509
5,0.13750,0.14262,0.14212,0.14539,0.14844,0.143214,0.003640
6,0.13702,0.14208,0.14223,0.14515,0.14842,0.142980,0.003774
7,0.13673,0.14196,0.14205,0.14532,0.14862,0.142936,0.003956
8,0.13685,0.14235,0.14228,0.14450,0.14815,0.142826,0.003671
9,0.13619,0.14224,0.14217,0.14465,0.14775,0.142600,0.003798


In [3]:
#train_100 = pd.read_csv('Airbnb_data/train_users_2.csv', nrows = 100, index_col='id')
train_data = pd.read_csv('data/train_users_2.csv', index_col = 'id')
test_data = pd.read_csv('data/test_users.csv', index_col = 'id')
#sessions = pd.read_csv('../data/sessions.csv')
#age = pd.read_csv('Airbnb_data/age_gender_bkts.csv')
#countries = pd.read_csv('Airbnb_data/countries.csv')
all_df = pd.read_pickle('data/all_df.p')

In [163]:
for i in np.unique(test_data.first_browser):
    if i not in np.unique(train_data.first_browser):
        print i

IBrowse
Nintendo Browser
UC Browser


In [142]:
#pd.isnull(sessions.groupby('user_id')['secs_elapsed'].sum()) == False
sessions.loc[sessions.user_id == '00allnceb8',:]
#sessions.loc[sessions.user_id == 'd1mm9tcy42',:]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
1150481,00allnceb8,create,submit,signup,-unknown-,NaN


In [8]:
#find most popular actions
sessions.groupby('user_id').count()

NameError: name 'sessions' is not defined

In [5]:
error = pd.read_pickle('scripts/cv_results.p')
params = pd.read_pickle('scripts/cv_params.p')

In [6]:
res = pd.concat([error, params], axis = 1)

In [7]:
res.sort_values(by = 'test-error-mean', axis = 0);

In [3]:
all_df = feakaggle_model(all_df)

#one hot encoding of features
print 'number of columns before one hot encoding', all_df.shape[1]
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
all_df = pd.get_dummies(all_df, columns = ohe_feats)
print 'number of columns after one hot encoding', all_df.shape[1]

number of columns before one hot encoding 17
number of columns after one hot encoding 161


In [6]:
with open('xgbmodels/sessions_e20_25n.p', 'rb') as f:
    bst = pickle.load(f)
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]

NameError: name 'all_df' is not defined

In [ ]:
xgb.to_graphviz(bst, num_trees=11)

In [ ]:
submission = pd.DataFrame(np.column_stack([idx, cntr]), columns = ['id', 'country'])
submission.to_csv('submissions/datebin_model.csv', index=False)